# Setup

In [1]:
import os

import tensorflow as tf
import tf_encrypted as tfe

Turn on TFE debugging flags first thing, optionally specifying where logging files should be written.

In [2]:
%load_ext tensorboard.notebook

TENSORBOARD_DIR = "/tmp/tensorboard"

tfe.set_tfe_trace_flag(True)
tfe.set_tfe_events_flag(True)
tfe.set_log_directory(TENSORBOARD_DIR)

INFO:tf_encrypted:Tensorflow encrypted is dumping computation traces
INFO:tf_encrypted:Writing event and trace files to '/tmp/tensorboard'


('Tensorflow encrypted is monitoring statistics for each', 'session.run() call using a tag')


# Computation

Define whatever computation we wish to inspect. Here we just do a few simple operations that TensorFlow can optimize: the `add` calls can be replaced with `add_n`.

In [3]:
x = tfe.define_private_input("input-provider", lambda: tf.fill([2,2], 2))

y = x + x + x

# Execution

We first remove previous event and tracing files to make it easier to find the new runs in TensorBoard.

In [4]:
!rm -rf {TENSORBOARD_DIR}

with tfe.Session(disable_optimizations=True) as sess:
    res = sess.run(y.reveal(), tag='res', output_partition_graphs=True)
    print(res)

INFO:tf_encrypted:Players: ['server0', 'server1', 'server2', 'input-provider']


[[6. 6.]
 [6. 6.]]


# Inspection

Let's first see what's in the logging directory.

In [5]:
!ls -l {TENSORBOARD_DIR}

total 40
drwxr-xr-x  6 dahl  wheel    192 May 21 14:21 res0
-rw-r--r--  1 dahl  wheel  20299 May 21 14:21 res0.ctr


Here we have a trace file, `res0.ctr`, for each tagged run, as well as a directory with event and partition files.

In [6]:
!ls -l {TENSORBOARD_DIR}/res0

total 104
-rw-r--r--  1 dahl  wheel  28178 May 21 14:21 events.out.tfevents.1558441303.WorkBook.local
-rw-r--r--  1 dahl  wheel  11510 May 21 14:21 partition0.pbtxt
-rw-r--r--  1 dahl  wheel   5007 May 21 14:21 partition1.pbtxt
-rw-r--r--  1 dahl  wheel   2108 May 21 14:21 partition2.pbtxt


All of these can be viewed using TensorBoard as shown below, but occasionally it's also useful to look directly at the partition protobuf files.

In [7]:
!cat {os.path.join(TENSORBOARD_DIR, "res0/partition0.pbtxt")}

node {
  name: "ConstantFolding/decode/truediv_recip"
  op: "Const"
  device: "/job:localhost/replica:0/task:0/device:CPU:0"
  attr {
    key: "dtype"
    value {
      type: DT_DOUBLE
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_DOUBLE
        tensor_shape {
        }
        tensor_content: "\225yT\2539\372#?"
      }
    }
  }
  experimental_debug_info {
    original_node_names: "ConstantFolding/decode/truediv_recip"
  }
}
node {
  name: "decode/sub/y"
  op: "Const"
  device: "/job:localhost/replica:0/task:0/device:CPU:0"
  attr {
    key: "dtype"
    value {
      type: DT_INT64
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_INT64
        tensor_shape {
        }
        int64_val: 14348907
      }
    }
  }
  experimental_debug_info {
    original_node_names: "decode/sub/y"
  }
}
node {
  name: "add_1/expand-seed/max"
  op: "Const"
  device: "/job:localhost/replica:0/task:0/device:CPU:0"
  attr {
    key: "dtype

You can load these files in TensorBoard via `Choose File`.

In [8]:
%tensorboard --logdir {TENSORBOARD_DIR}

Reusing TensorBoard on port 6010 (pid 64858), started -1 day, 23:06:21 ago. (Use '!kill 64858' to kill it.)